In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten , Conv2D, Dropout, BatchNormalization, ReLU
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
import time
from math import trunc
import matplotlib.pyplot as plt

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()

# Ajout de l'indication du canal sur le images pour le traitement via réseau convolutionnel
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)

input_shape=(28, 28, 1)
num_classes = 10

# normalisation
x_train, x_val = x_train/255., x_val/255.
# 1-hot encoding
y_train, y_val = to_categorical(y_train, dtype='int8'), to_categorical(y_val, dtype='int8')

In [ ]:
# Hyperparamètres
#
# Nombre d'époques pour l'apprentissage:
epochs = 30

# Taille du batch d'entraînement:
batch_size = 100

# Taux de dropout rate
dropout_rate = 0.1

# Planification de l'apprentissage
initial_learning_rate = 0.005
# taux de décroissance du pas d'apprentissage
decay_rate = 0.7
# Nombre d'époques à faire avant de changer de pas
decay_steps = 180000 # 3 époques

In [ ]:
# Structure du modèle

model = Sequential()
model.add(Conv2D(12, kernel_size=(6,6), input_shape=input_shape))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(dropout_rate))

model.add(Conv2D(12, (5,5), strides=(2, 2)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(dropout_rate))

model.add(Conv2D(24, (4,4), strides=(2, 2)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(dropout_rate))

model.add(Flatten())

model.add(Dense(200, activation='relu'))
model.add(Dropout(dropout_rate))

model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# Planification de l'apprentissage

lr_schedule = ExponentialDecay(initial_learning_rate,
            decay_steps = decay_steps,
            decay_rate = decay_rate,
            staircase=True)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
              metrics = ['accuracy'])

In [ ]:
heure_de_debut = time.time()
historique = model.fit(x_train, y_train,
                       batch_size=batch_size,
                       epochs=epochs,
                       verbose=1,
                       validation_data=(x_val, y_val))

temps_de_calcul = time.time() - heure_de_debut
print("Temps de calcul: {:d} s".format(trunc(temps_de_calcul)))

In [ ]:
model.evaluate(x_val, y_val, verbose=0)

In [ ]:
# Représentation des résultats
accuracy_train = historique.history['accuracy']
accuracy_test = historique.history['val_accuracy']
loss_train = historique.history['loss']
loss_test = historique.history['val_loss']
# Précision
plt.plot(accuracy_test, c='coral', label='test')
plt.plot(accuracy_train, c='steelblue', label='val')
plt.xlabel('Epoque')
plt.title('Précision')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
# Fonction de coût
plt.plot(loss_test, c='coral', label='test')
plt.plot(loss_train, c='steelblue', label='val')
plt.title('Fonction de coût')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()